In [19]:
import cobra
import os
import pandas as pd
import numpy as np
import urllib
from cobrame.util import building, mu, me_model_interface

## Metabolites

met_output_file = 'metabolites.txt'
m_model_file = './iYO844.json'

m_model = cobra.io.load_json_model(m_model_file)

In [20]:
def duplicates(lst, item, counter):
    return [i for i, x in enumerate(lst) if x == item and i != counter]

def duplicate_indices(lst, item):
    return [i for i, x in enumerate(lst) if x == item]

def convert_compartments(compartment_list):
    compartment_ids = ['c','e','p']
    compartment_equivalence = ['Cytosol','Extra-organism','Periplasm']
    
    for c in range(0,len(compartment_list)):
        if compartment_list[c] in compartment_ids:
            ind = compartment_ids.index(compartment_list[c])
            compartment_list[c] = compartment_equivalence[ind]
            
    return compartment_list

def convert_compartment(compartment_list):
    compartment_ids = ['c','e','p']
    compartment_equivalence = ['Cytosol','Extra-organism','Periplasm']
    if compartment_list in compartment_ids:
        ind = compartment_ids.index(compartment_list)
        compartment_list = compartment_equivalence[ind]
            
    return compartment_list

## 1. metabolites.txt

In [21]:
def generate_metfile(m_model,filename):
    file = open(filename,'w')
    
    metabolite_list = [None]*len(m_model.metabolites)
    compartment_list = [None]*len(m_model.metabolites)
    
    for m in range(0,len(m_model.metabolites)):
        ID = m_model.metabolites[m].id
        compartment_list[m] = ID[len(ID)-1]
        metabolite_list[m] = ID[:len(ID)-2]
        
    compartment_list = convert_compartments(compartment_list)
    
    other_mets_index = [];
    for c in range(0,len(m_model.metabolites)):
        if len(duplicates(other_mets_index,c,-1)) == 0:
            met = metabolite_list[c]
            name = m_model.metabolites[c].name
            formula = m_model.metabolites[c].formula

            other_mets_index = duplicates(metabolite_list,met,c)
            
            if formula is None:
                formula = 'no-formula'
            
            string = met + '\t' + name + '\t' + formula + '\t' + compartment_list[c]
            for other_mets in other_mets_index:
                string = string + ' AND ' + compartment_list[other_mets]
            
            string = string + '\n'
            file.write(string)
    file.close()

In [22]:
generate_metfile(m_model,met_output_file)

## reaction_matrix.txt

In [23]:
def generate_reactionmatrix_file(m_model,filename):
    file = open(filename,'w')
    
    reaction_matrix_list = []
    
    for rxn in m_model.reactions:
        for metabolite in rxn.metabolites:
            compartment = metabolite.compartment
            compartment = convert_compartment(compartment)
            
            met = metabolite.id
            met = met[:len(met)-2]
            
            coefficient = rxn.get_coefficient(metabolite)
            
            string = rxn.id + '\t' + met + '\t' + compartment + '\t' + str(coefficient) + '\n'
            
            file.write(string)

    file.close()
    
    
    

In [24]:
generate_reactionmatrix_file(m_model,'reaction_matrix.txt')

## reactions.txt

In [27]:
def generate_reactions_file(m_model,filename):
    file = open(filename,'w')
    
    string = '#name' + '\t' + 'description' +  '\t' + 'is_reversible' + '\t' + 'data_source' + '\t' + 'is_spontaneous' +  '\n'
    
    file.write(string)
    reaction_matrix_list = []
    
    for rxn in m_model.reactions:
        rev = rxn.reversibility
        name = rxn.name
        name = name.replace(' ','_')
        
        if rev == True:
            reversibility = 1
        else:
            reversibility = 0
        
        string = rxn.id + '\t' + name + '\t' + str(reversibility) + '\t' + m_model.id + '\t' + '0' + '\n'
        file.write(string)

    file.close()

In [28]:
generate_reactions_file(m_model,'reactions.txt')

In [34]:
from Bio.Data import CodonTable
print(Bio.Data.CodonTable.)

TypeError: 'dict' object is not callable

In [30]:
Bio.Data.CodonTable.standard_rna_table

Table 1 Standard, SGC0

  |  U      |  C      |  A      |  G      |
--+---------+---------+---------+---------+--
U | UUU F   | UCU S   | UAU Y   | UGU C   | U
U | UUC F   | UCC S   | UAC Y   | UGC C   | C
U | UUA L   | UCA S   | UAA Stop| UGA Stop| A
U | UUG L(s)| UCG S   | UAG Stop| UGG W   | G
--+---------+---------+---------+---------+--
C | CUU L   | CCU P   | CAU H   | CGU R   | U
C | CUC L   | CCC P   | CAC H   | CGC R   | C
C | CUA L   | CCA P   | CAA Q   | CGA R   | A
C | CUG L(s)| CCG P   | CAG Q   | CGG R   | G
--+---------+---------+---------+---------+--
A | AUU I   | ACU T   | AAU N   | AGU S   | U
A | AUC I   | ACC T   | AAC N   | AGC S   | C
A | AUA I   | ACA T   | AAA K   | AGA R   | A
A | AUG M(s)| ACG T   | AAG K   | AGG R   | G
--+---------+---------+---------+---------+--
G | GUU V   | GCU A   | GAU D   | GGU G   | U
G | GUC V   | GCC A   | GAC D   | GGC G   | C
G | GUA V   | GCA A   | GAA E   | GGA G   | A
G | GUG V   | GCG A   | GAG E   | GGG G   | G
--+---------